In [ ]:
import os
from pprint import pprint
import re
import warnings
from deepdiff import DeepDiff
import openpyxl
import pandas as pd

from compare import append_to_dict, dict_to_table, save_df_to_excel


def light_bom_excel_to_dictionary(filePath: str):
    warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

    # Load the workbook and select the sheet
    wb = openpyxl.load_workbook(filePath, data_only=True)
    ws = wb['Bom']

    # Find the PivotTable named 'BomOracle'
    pivot = None
    for pt in ws._pivots:
        if pt.name == 'BomOracle':
            pivot = pt
            break

    if pivot is None:
        raise ValueError("PivotTable 'BomOracle' not found.")

    # Get the pivot table's displayed range (where it is rendered in the sheet)
    pivot_display_range = pivot.location.ref

    # Extract the displayed data from the range
    data = []
    for row in ws[pivot_display_range]:
        data.append([cell.value for cell in row])

    # Convert to pandas DataFrame
    df = pd.DataFrame(data[1:], columns=data[0])
    df = df[df['Item'].notna()].reset_index(drop=True)
    # Remove suffix pattern from 'Item' column
    df['Item'] = df['Item'].str.replace(r'_\d{2}$', '', regex=True)

    # Rename columns
    df = df.rename(columns={'Supplier_Type': 'SupplyType', 'Item': 'Item name','SE_REVISION':'Revision','Qty':'Quantity' })

    # Keep only columns that are different from the original and not unchanged
    df = df[['SupplyType', 'Item name', 'Revision', 'Quantity', 'Description']]

    # Add 'Depth' column full of 1
    df['Depth'] = 1

    light_bom = {}

    for _, row in df.iterrows():
        item = row['Item name']
        light_bom[item] = row.to_dict()

    return light_bom

bom1 = light_bom_excel_to_dictionary("C:/Users/SESA787052/Downloads/BOM QBOT21000 _rev03.xlsx")
bom2 = light_bom_excel_to_dictionary("C:/Users/SESA787052/Downloads/BOM QBOT21000 _rev03-02.xlsx")

diff = DeepDiff(bom1, bom2, threshold_to_diff_deeper=0)
max_depth = 1

item_added = [re.findall(r'\[\'(.*?)\'\]', element.replace('[\'content\']', "").replace("root", ""))
                for element in diff.get('dictionary_item_added', [])]
item_removed = [re.findall(r'\[\'(.*?)\'\]', element.replace('[\'content\']', "").replace("root", ""))
                for element in diff.get('dictionary_item_removed', [])]
item_changed = [(re.findall(r'\[\'(.*?)\'\]', element.replace('[\'content\']', "").replace("root", "")), diff['values_changed'][element])
                for element in diff.get('values_changed', [])]

output = {}

for item in item_added:
    output = append_to_dict(item, bom2, {'type': 'ADDED'}, output)

for item in item_removed:
    output = append_to_dict(item, bom1, {'type': 'REMOVED'}, output)

print(item_changed)

for item in item_changed:
    output = append_to_dict(item[0][:-1], bom1, {'type': 'CHANGED', 'changed_value': item[0][-1], **item[1]}, output)

table_output = dict_to_table(output, max_depth)
columns = ['Level', *[str(i) for i in range(1, max_depth+1)], 'Item', 'Description', 'Revision',
            'Quantity', 'SupplyType', 'ModifyType']

output_df = pd.DataFrame(table_output, columns=columns)
output_path = f"C:\\Users\\{os.getlogin()}\\Downloads\\compare_result.xlsx"


if item_added or item_changed or item_removed:
    save_df_to_excel(output_df, max_depth, output_path)
else:
    print("Nothing changed")

In [8]:
import warnings
def find_cost_of(search_string: str):
    """
    Finds the unit cost of an item in a Bill of Materials (BOM) Excel file based on a search string.

    Args:
        search_string (str): The string to search for in the 'Item' column of the BOM.

    Returns:
        pandas.Series: A series containing the unit cost(s) of the item(s) that match the search string.
    """

    filePath = 'C:/Users/SESA787052/Documents/GVXL/BOM Valorisée GVXLIOCQ-QASH01250-PROTO.xlsx'
    
    with warnings.catch_warnings(action="ignore"):
        skip = 0
        BOM = pd.read_excel(filePath, skiprows=skip)
        result_row = BOM[BOM['Item'].str.contains(search_string, na=False)]
        return result_row['Unit Cost']
    


10    67.77
Name: Unit Cost, dtype: float64


In [1]:
from compare import compare_bom
from pprint import pprint

bom1 = "C:/OneDrive - Schneider Electric/Documents - MBT Plant - Gestion affaires - Métiers Group/Affaires/2025/706585418/GVXLIOCQ-QONE01250-00/Prod/03-PVZ, Schéma Elec, Specif et BOM/Spécifications/BOM QONE01250.xlsx"
bom2 = "C:/OneDrive - Schneider Electric/Documents - MBT Plant - Gestion affaires - Métiers Group/Affaires/2025/706585418/GVXLIOCQ-QONE01250-00/Prod/03-PVZ, Schéma Elec, Specif et BOM/Spécifications/BOM QONE01250_REV02.xlsx"

compare_bom(bom1, bom2, open_result=True)

c:\Users\SESA787052\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\reader\workbook.py:118: UserWarning: Print area cannot be set to Defined name: Tableau2[].
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
c:\Users\SESA787052\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\reader\workbook.py:118: UserWarning: Print area cannot be set to Defined name: Tableau2[].
  warn(f"Print area cannot be set to Defined name: {defn.value}.")


[['GVXL_PE_BARS_EXTRA_ROOF_TO_UPS'], ['GVXL_DC_EXTRA_ROOF', '34413584', '34413589_SCR', '34413589_02'], ['GVXL_DC_EXTRA_ROOF', '34413584', '34413588_SCR', '34413588_02']]
[['34413610_SCR'], ['GVXL_DC_EXTRA_ROOF', '34413584', '34413589_SCR', '34413589_01'], ['GVXL_DC_EXTRA_ROOF', '34413584', '34413588_SCR', '34413588_01']]
[(['GVXL_DC_EXTRA_ROOF', '34413584', '34413589_SCR', 'Revision'], {'new_value': 2, 'old_value': 1}), (['GVXL_DC_EXTRA_ROOF', '34413584', '34413588_SCR', 'Revision'], {'new_value': 2, 'old_value': 1})]


{'Item name': 'GVXL_PE_BARS_EXTRA_ROOF_TO_UPS', 'Revision': 'XX', 'Description': 'PE bars from UPS to dc extra roof', 'Quantity': 1, 'SupplyType': nan, 'Depth': 1, 'content': {'34413722_01': {'Item name': '34413722_01', 'Revision': 1, 'Description': 'PE BAR', 'Quantity': 2, 'SupplyType': 'CUIVRE', 'Depth': 2, 'content': {}}, 'HUA12375': {'Item name': 'HUA12375', 'Revision': 'XX', 'Description': 'HUA12375 - Screw MSC HXG ISO4017-M10X40-A2K-8_8 ou VDZ STL', 'Quantity': 4

ValueError: 12 columns passed, passed data had 14 columns